# Gun Violence in U.S.A Analysis

# *The link of dataset

https://www.kaggle.com/jameslko/gun-violence-data

## 1. PySpark environment setup

In [188]:
import findspark
findspark.init()

from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

sc = SparkContext.getOrCreate()
spark = SparkSession(sc)

## 2. Data source and Spark data abstraction (DataFrame) setup

In [187]:
GunDF = gun_violence_data.csv

NameError: name 'gun_violence_data' is not defined

In [190]:
GunDF = spark.read \
                 .option("inferSchema", "true") \
                 .option("header", "true") \
                 .csv("gun_violence_data.csv")
GunDF

DataFrame[incident_id: string, date: string, state: string, city_or_county: string, address: string, n_killed: string, n_injured: string, incident_url: string, source_url: string, incident_url_fields_missing: string, congressional_district: string, gun_stolen: string, gun_type: string, incident_characteristics: string, latitude: string, location_description: string, longitude: double, n_guns_involved: int, notes: string, participant_age: string, participant_age_group: string, participant_gender: string, participant_name: string, participant_relationship: string, participant_status: string, participant_type: string, sources: string, state_house_district: string, state_senate_district: string]

## 3. Data set metadata analysis
### A. Display schema and size of the DataFrame

In [191]:
from IPython.display import display, Markdown

GunDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % GunDF.count()))

root
 |-- incident_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city_or_county: string (nullable = true)
 |-- address: string (nullable = true)
 |-- n_killed: string (nullable = true)
 |-- n_injured: string (nullable = true)
 |-- incident_url: string (nullable = true)
 |-- source_url: string (nullable = true)
 |-- incident_url_fields_missing: string (nullable = true)
 |-- congressional_district: string (nullable = true)
 |-- gun_stolen: string (nullable = true)
 |-- gun_type: string (nullable = true)
 |-- incident_characteristics: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- location_description: string (nullable = true)
 |-- longitude: double (nullable = true)
 |-- n_guns_involved: integer (nullable = true)
 |-- notes: string (nullable = true)
 |-- participant_age: string (nullable = true)
 |-- participant_age_group: string (nullable = true)
 |-- participant_gender: string (nullable = true)
 |-- par

This DataFrame has **246939 rows**.

### B. Get one or multiple random samples from the data set

In [192]:
GunDF.cache() # optimization to make the processing faster
GunDF.sample(False, 0.1).take(2)

[Row(incident_id='478948', date='2013-01-07', state='Oklahoma', city_or_county='Tulsa', address='6000 block of South Owasso', n_killed='4', n_injured='0', incident_url='http://www.gunviolencearchive.org/incident/478948', source_url='http://usnews.nbcnews.com/_news/2013/01/07/16397584-police-four-women-found-dead-in-tulsa-okla-apartment?lite', incident_url_fields_missing='False', congressional_district='1', gun_stolen=None, gun_type=None, incident_characteristics='Shot - Dead (murder, accidental, suicide)||Home Invasion||Home Invasion - Resident killed||Mass Shooting (4+ victims injured or killed excluding the subject/suspect/perpetrator, one location)||Armed robbery with injury/death and/or evidence of DGU found', latitude='36.2405', location_description='Fairmont Terrace', longitude=-95.9768, n_guns_involved=None, notes=None, participant_age='0::23||1::23||2::33||3::55', participant_age_group='0::Adult 18+||1::Adult 18+||2::Adult 18+||3::Adult 18+||4::Adult 18+||5::Adult 18+', partici

## 4. Data Cleaning

### A. Check missing value percentage in every column

In [193]:
import pyspark.sql.functions as fn
GunDF.agg(*[(1-(fn.count(c) /fn.count('*'))).alias(c+'_missing') for c in GunDF.columns]).show()

+-------------------+--------------------+--------------------+----------------------+------------------+--------------------+--------------------+--------------------+--------------------+-----------------------------------+------------------------------+-------------------+-------------------+--------------------------------+--------------------+----------------------------+--------------------+-----------------------+-----------------+-----------------------+-----------------------------+--------------------------+------------------------+--------------------------------+--------------------------+------------------------+------------------+----------------------------+-----------------------------+
|incident_id_missing|        date_missing|       state_missing|city_or_county_missing|   address_missing|    n_killed_missing|   n_injured_missing|incident_url_missing|  source_url_missing|incident_url_fields_missing_missing|congressional_district_missing| gun_stolen_missing|   gun_type_

### B. Drop the columns which missing value are more than 15% and irrelevant to our analysis

In [194]:
columns_to_drop = ['address', 'incident_url', 'source_url', 'incident_url_fields_missing', 'congressional_district', 'gun_stolen', 'gun_type', 'latitude', 'location_description', 'longitude', 'n_guns_involved', 'notes', 'participant_age', 'incident_characteristics', 'participant_age_group', 'participant_gender', 'participant_name', 'participant_relationship', 'participant_status', 'participant_type', 'sources', 'state_house_district', 'state_senate_district']
GunDF = GunDF.drop(*columns_to_drop)
print("Our data have", len(GunDF.columns), "columns")
GunDF.printSchema()
display(Markdown("This DataFrame has **%d rows**." % GunDF.count()))

Our data have 6 columns
root
 |-- incident_id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- state: string (nullable = true)
 |-- city_or_county: string (nullable = true)
 |-- n_killed: string (nullable = true)
 |-- n_injured: string (nullable = true)



This DataFrame has **246939 rows**.

### C. Check missing value for the rest of the columns

In [358]:
print("Checking for nulls on columns incident_id, date, state, city_or_county, n_killed, n_injured:")
GunDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["incident_id", "date", "state", "city_or_county", "n_killed", "n_injured"]]).show()

Checking for nulls on columns incident_id, date, state, city_or_county, n_killed, n_injured:
+-----------+----+-----+--------------+--------+---------+
|incident_id|date|state|city_or_county|n_killed|n_injured|
+-----------+----+-----+--------------+--------+---------+
|          0|   0|    0|             0|    3238|      914|
+-----------+----+-----+--------------+--------+---------+



### D. Deal with missing values -- 'state' and 'city_or_county'

In [196]:
# 1.State and city_or_county are two main analytical criteria, for our dataset, the geographical attribute is better to remove rather than fill it. 
#so we delete the row which contain null value in these two columns.
GunDF = GunDF.filter(GunDF.state. isNotNull())
GunDF = GunDF.filter(GunDF.city_or_county. isNotNull())
GunDF = GunDF.filter(GunDF.date. isNotNull())


#Check if it works
print("Checking for nulls on columns incident_id, date, state, city_or_county, n_killed, n_injured:")
GunDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["incident_id", "date", "state", "city_or_county", "n_killed", "n_injured"]]).show()

Checking for nulls on columns incident_id, date, state, city_or_county, n_killed, n_injured:
+-----------+----+-----+--------------+--------+---------+
|incident_id|date|state|city_or_county|n_killed|n_injured|
+-----------+----+-----+--------------+--------+---------+
|          0|   0|    0|             0|    1382|     3706|
+-----------+----+-----+--------------+--------+---------+



### E. Deal with missing values -- 'n_killed' and 'n_injured'

In [197]:
# 2.for n_killed and n_injured, we fill "mode = 0" into all null value
# So first we need to calculate the mode for these two cloumns

GunDF = GunDF.fillna({"n_killed":0, "n_injured":0})

# check result
print("Checking for nulls on columns incident_id, date, state, city_or_county, n_killed, n_injured:")
GunDF.select([count(when(col(c).isNull(), c)).alias(c) for c in ["incident_id", "date", "state", "city_or_county", "n_killed", "n_injured"]]).show()

Checking for nulls on columns incident_id, date, state, city_or_county, n_killed, n_injured:
+-----------+----+-----+--------------+--------+---------+
|incident_id|date|state|city_or_county|n_killed|n_injured|
+-----------+----+-----+--------------+--------+---------+
|          0|   0|    0|             0|       0|        0|
+-----------+----+-----+--------------+--------+---------+



## 5. Column basic profiling

### A. Switch datatype of "n_killed" and "n_injured" from str to int

In [198]:
# Column n_injured and n_killed are both str datatype in the beginning, for further calculation we have to swith them into int

def get_dtype(GunDF,n_injured):
    return [dtype for name, dtype in GunDF.dtypes if name == n_injured][0]

get_dtype(GunDF,'n_injured')


'string'

In [253]:
from pyspark.sql.types import IntegerType
GunDF = GunDF.withColumn("n_killed", GunDF["n_killed"].cast(IntegerType()))
GunDF = GunDF.withColumn("n_injured", GunDF["n_injured"].cast(IntegerType()))

#test
get_dtype(GunDF,'n_injured')

'int'

### B. Get summary profile

In [200]:
from IPython.display import display, Markdown
from pyspark.sql.functions import when, count, col, countDistinct, desc, first

print ("Summary of columns date:")
GunDF.select("date").summary().show()

print("Checking amount of distinct values in columns date:")
GunDF.select([countDistinct(c).alias(c) for c in ["date"]]).show()

print ("Summary of columns state, city_or_county:")
GunDF.select("state", "city_or_county").summary().show()

print("Checking amount of distinct values in columns state, city_or_county:")
GunDF.select([countDistinct(c).alias(c) for c in ["state", "city_or_county"]]).show()

print ("Summary of columns n_killed, n_injured:")
GunDF.select("n_killed", "n_injured").summary().show()

print("Checking amount of distinct values in columns n_killed, n_injured:")
GunDF.select([countDistinct(c).alias(c) for c in ["n_killed", "n_injured"]]).show()

Summary of columns date:
+-------+-----------------+
|summary|             date|
+-------+-----------------+
|  count|           244297|
|   mean|     -62.25280375|
| stddev|95.04325055653644|
|    min|   -100.011119 ;"|
|    25%|      -149.817365|
|    50%|       -87.019119|
|    75%|       -85.174731|
|    max|            4::59|
+-------+-----------------+

Checking amount of distinct values in columns date:
+----+
|date|
+----+
|6210|
+----+

Summary of columns state, city_or_county:
+-------+--------------------+--------------------+
|summary|               state|      city_or_county|
+-------+--------------------+--------------------+
|  count|              244297|              244297|
|   mean|                22.0|                null|
| stddev|                 NaN|                null|
|    min| -75.462676 did g...| 9mm and .22-cali...|
|    25%|                22.0|                null|
|    50%|                22.0|                null|
|    75%|                22.0|          

## 6. Analysis 1_State

### A. Calculate each state with their total incident separately

In [247]:
GunGeoDF = \
   GunDF.groupBy("state")\
        .agg(count("incident_id").alias("totalincident"))\
        .orderBy(col("totalincident").desc())

GunGeoDF.limit(20).show()

+--------------+-------------+
|         state|totalincident|
+--------------+-------------+
|      Illinois|        17556|
|    California|        16306|
|       Florida|        15029|
|         Texas|        13577|
|          Ohio|        10244|
|      New York|         9712|
|  Pennsylvania|         8929|
|       Georgia|         8925|
|North Carolina|         8739|
|     Louisiana|         8103|
|     Tennessee|         7626|
|South Carolina|         6939|
|      Missouri|         6631|
|      Michigan|         6136|
| Massachusetts|         5981|
|      Virginia|         5949|
|       Indiana|         5852|
|      Maryland|         5798|
|       Alabama|         5471|
|    New Jersey|         5387|
+--------------+-------------+



### B. Create alert level for different amount of gun violence incident

In [352]:
RatioGunGeoDF = GunGeoDF\
            .withColumn("GunViolenceSeverityLevel", when(col("totalincident")>=10000,"Black Alert")\
            .when((col("totalincident")>=8000) & (col("totalincident")<=9999),"Red Alert")\
            .when((col("totalincident")>=6000) & (col("totalincident")<=7999),"Yellow Alert")\
            .when((col("totalincident")>=2000) & (col("totalincident")<=5999),"Blue Alert")\
            .otherwise("Green Alert"))

RatioGunGeoDF.limit(20).show()

+--------------+-------------+------------------------+
|         state|totalincident|GunViolenceSeverityLevel|
+--------------+-------------+------------------------+
|      Illinois|        17556|             Black Alert|
|    California|        16306|             Black Alert|
|       Florida|        15029|             Black Alert|
|         Texas|        13577|             Black Alert|
|          Ohio|        10244|             Black Alert|
|      New York|         9712|               Red Alert|
|  Pennsylvania|         8929|               Red Alert|
|       Georgia|         8925|               Red Alert|
|North Carolina|         8739|               Red Alert|
|     Louisiana|         8103|               Red Alert|
|     Tennessee|         7626|            Yellow Alert|
|South Carolina|         6939|            Yellow Alert|
|      Missouri|         6631|            Yellow Alert|
|      Michigan|         6136|            Yellow Alert|
| Massachusetts|         5981|              Blue

### C. Select the most serious level- Black alert for the following analysis

In [259]:
BlackalertDF = \
  RatioGunGeoDF.where((col("GunViolenceSeverityLevel")=="Black Alert"))\

BlackalertDF.show()

+----------+-------------+------------------------+
|     state|totalincident|GunViolenceSeverityLevel|
+----------+-------------+------------------------+
|  Illinois|        17556|             Black Alert|
|California|        16306|             Black Alert|
|   Florida|        15029|             Black Alert|
|     Texas|        13577|             Black Alert|
|      Ohio|        10244|             Black Alert|
+----------+-------------+------------------------+



## 7. Analysis 2_Black alert state deep analysis - Illinois as an example

### A. Get deeper in states with black alert - city or county

In [345]:
#Illinois state

IlliDF = GunDF.where(col("state")=="Illinois")\
                .select("state", "city_or_county")\
                .groupBy("state", "city_or_county")\
                .agg(count("city_or_county").alias("totalincident"))\
                .sort(F.col("totalincident").desc())\

IlliDF.limit(5).show()

#California state

CaliDF = GunDF.where(col("state")=="California")\
                .select("state", "city_or_county")\
                .groupBy("state", "city_or_county")\
                .agg(count("city_or_county").alias("totalincident"))\
                .sort(F.col("totalincident").desc())\
                .withColumnRenamed("city_or_county", "California_city_or_county")

CaliDF.limit(5).show()



#Florida state

FloDF = GunDF.where(col("state")=="Florida")\
                .select("state", "city_or_county")\
                .groupBy("state", "city_or_county")\
                .agg(count("city_or_county").alias("totalincident"))\
                .sort(F.col("totalincident").desc())\
                .withColumnRenamed("city_or_county", "Florida_city_or_county")

FloDF.limit(5).show()
#Texas state

TexDF = GunDF.where(col("state")=="Texas")\
                .select("state", "city_or_county")\
                .groupBy("state", "city_or_county")\
                .agg(count("city_or_county").alias("totalincident"))\
                .sort(F.col("totalincident").desc())\
                .withColumnRenamed("city_or_county", "Texas_city_or_county")

TexDF.limit(5).show()

#Ohio state

OhioDF = GunDF.where(col("state")=="Ohio")\
                .select("state", "city_or_county")\
                .groupBy("state", "city_or_county")\
                .agg(count("city_or_county").alias("totalincident"))\
                .sort(F.col("totalincident").desc())\
                .withColumnRenamed("city_or_county", "Ohio_city_or_county")

OhioDF.limit(5).show()

#Now we can see which city or county needs to be focus primarily

+--------+-------------------+-------------+
|   state|     city_or_county|totalincident|
+--------+-------------------+-------------+
|Illinois|            Chicago|        10814|
|Illinois|             Peoria|          920|
|Illinois|           Rockford|          842|
|Illinois|Chicago (Englewood)|          542|
|Illinois|        Springfield|          303|
+--------+-------------------+-------------+

+----------+-------------------------+-------------+
|     state|California_city_or_county|totalincident|
+----------+-------------------------+-------------+
|California|                  Oakland|         1478|
|California|              Los Angeles|         1066|
|California|                   Fresno|         1057|
|California|              Bakersfield|          605|
|California|                 Stockton|          555|
+----------+-------------------------+-------------+

+-------+----------------------+-------------+
|  state|Florida_city_or_county|totalincident|
+-------+-------------

### B. Average of people get killed in city or county - Illinois as an example

In [332]:
from pyspark.sql.functions import max, min, avg, stddev
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import count, round

KilledIlliDF = GunDF.where(col("state")=="Illinois")\
                .select("state", "city_or_county", "n_killed")\
                .groupBy("state", "city_or_county")\
                .agg(avg("n_killed").alias("averagekilled"))\
                .sort(F.col("averagekilled").desc())\
                .select("state", "city_or_county",round("averagekilled",2).alias("averagekilled"))

KilledIlliDF.show()
              

+--------+------------------+-------------+
|   state|    city_or_county|averagekilled|
+--------+------------------+-------------+
|Illinois|        Manchester|          6.0|
|Illinois|            Dwight|          3.0|
|Illinois|           Geneseo|          2.5|
|Illinois|          Danforth|          2.0|
|Illinois|  Bay View Gardens|          2.0|
|Illinois|         Braidwood|          2.0|
|Illinois|            Warsaw|          2.0|
|Illinois|          Columbia|          2.0|
|Illinois|Benton (West City)|          2.0|
|Illinois|      Morton Grove|          2.0|
|Illinois|              Avon|          2.0|
|Illinois|           Oakwood|          2.0|
|Illinois|       Hazel Crest|         1.67|
|Illinois|     Cherry Valley|          1.5|
|Illinois|      River Forest|          1.5|
|Illinois|      Bloomingdale|          1.5|
|Illinois|      Ford Heights|          1.4|
|Illinois|  Round Lake Beach|          1.0|
|Illinois|         Northlake|          1.0|
|Illinois|           Lynwood|   

### C. Average of people get injgured in city or county - Illinois as an example

In [355]:
InjuredIlliDF = GunDF.where(col("state")=="Illinois")\
                .select("state", "city_or_county", "n_injured")\
                .groupBy("state", "city_or_county")\
                .agg(avg("n_injured").alias("averageinjured"))\
                .sort(F.col("averageinjured").desc())\
                .select("state","city_or_county",round("averageinjured",2).alias("averageinjured"))

InjuredIlliDF.show()


+--------+--------------------+--------------+
|   state|      city_or_county|averageinjured|
+--------+--------------------+--------------+
|Illinois|         Centreville|           5.0|
|Illinois|          Hanna City|           3.0|
|Illinois|          Lake Bluff|           3.0|
|Illinois|        Calumet Park|           2.0|
|Illinois|    Mchenry (county)|           2.0|
|Illinois|         Carlinville|           2.0|
|Illinois|              Oreana|           2.0|
|Illinois|            Bellmont|           2.0|
|Illinois|    Vermilion County|           2.0|
|Illinois|        Hopkins Park|           2.0|
|Illinois|            Homewood|           2.0|
|Illinois|             Momence|          1.67|
|Illinois|              Rankin|          1.67|
|Illinois|  Chicago (Norridge)|           1.5|
|Illinois|              Venice|          1.43|
|Illinois|  Lovejoy (Brooklyn)|           1.4|
|Illinois|              Mounds|          1.33|
|Illinois|        Sauk Village|          1.33|
|Illinois|   

### D. Merge table together - Illinois as an example

In [337]:
# merge InjuredIlliDF and KilledIlliDF first

kandiDF = InjuredIlliDF.join(KilledIlliDF, on=["city_or_county", "state"]).select("state", "city_or_county", "averageinjured", "averagekilled")
kandiDF.show()

+--------+-------------------+--------------+-------------+
|   state|     city_or_county|averageinjured|averagekilled|
+--------+-------------------+--------------+-------------+
|Illinois|            Addison|           0.5|          1.0|
|Illinois|          Braidwood|           0.0|          2.0|
|Illinois|Chicago (Englewood)|          1.01|         0.26|
|Illinois|           Lockport|          0.27|         0.27|
|Illinois|             Dekalb|          0.11|         0.22|
|Illinois|   Kendall (county)|           0.0|          0.0|
|Illinois|       Libertyville|           0.0|          0.5|
|Illinois|           O'fallon|           0.0|          0.0|
|Illinois|         Homer Glen|           0.5|          1.0|
|Illinois|       Melrose Park|          0.43|         0.71|
|Illinois|         New Boston|           0.0|          0.0|
|Illinois|             Quincy|          0.29|         0.08|
|Illinois|         Crest Hill|           0.0|          0.0|
|Illinois|        Lynn Center|          

In [351]:
# merge kandiDF and IlliDF with Gun Violence Severity Level indication

allDF = IlliDF.join(kandiDF, on=["city_or_county", "state"]).select("state", "city_or_county", "totalincident", "averageinjured", "averagekilled")
allDF.sort(F.col("totalincident").desc())\
            .withColumn("GunViolenceSeverityLevel", when(col("totalincident")>=10000,"Black Alert")\
            .when((col("totalincident")>=8000) & (col("totalincident")<=9999),"Red Alert")\
            .when((col("totalincident")>=6000) & (col("totalincident")<=7999),"Yellow Alert")\
            .when((col("totalincident")>=2000) & (col("totalincident")<=5999),"Blue Alert")\
            .otherwise("Green Alert")).show()

# By using Illinois as an example, the other states be done with the same code and method.

+--------+--------------------+-------------+--------------+-------------+------------------------+
|   state|      city_or_county|totalincident|averageinjured|averagekilled|GunViolenceSeverityLevel|
+--------+--------------------+-------------+--------------+-------------+------------------------+
|Illinois|             Chicago|        10814|          0.96|         0.19|             Black Alert|
|Illinois|              Peoria|          920|          0.32|         0.04|             Green Alert|
|Illinois|            Rockford|          842|          0.47|         0.09|             Green Alert|
|Illinois| Chicago (Englewood)|          542|          1.01|         0.26|             Green Alert|
|Illinois|         Springfield|          303|          0.37|         0.13|             Green Alert|
|Illinois|           Champaign|          213|          0.38|         0.12|             Green Alert|
|Illinois|              Joliet|          198|          0.46|         0.23|             Green Alert|


In [357]:
allDF.sort(F.col("averageinjured").desc())\
            .withColumn("GunViolenceSeverityLevel", when(col("totalincident")>=10000,"Black Alert")\
            .when((col("totalincident")>=8000) & (col("totalincident")<=9999),"Red Alert")\
            .when((col("totalincident")>=6000) & (col("totalincident")<=7999),"Yellow Alert")\
            .when((col("totalincident")>=2000) & (col("totalincident")<=5999),"Blue Alert")\
            .otherwise("Green Alert")).show()

+--------+-------------------+-------------+--------------+-------------+------------------------+
|   state|     city_or_county|totalincident|averageinjured|averagekilled|GunViolenceSeverityLevel|
+--------+-------------------+-------------+--------------+-------------+------------------------+
|Illinois|        Centreville|            1|           5.0|          0.0|             Green Alert|
|Illinois|         Lake Bluff|            1|           3.0|          0.0|             Green Alert|
|Illinois|         Hanna City|            1|           3.0|          0.0|             Green Alert|
|Illinois|       Hopkins Park|            1|           2.0|          0.0|             Green Alert|
|Illinois|        Carlinville|            1|           2.0|          0.0|             Green Alert|
|Illinois|           Homewood|            1|           2.0|          0.0|             Green Alert|
|Illinois|   Mchenry (county)|            1|           2.0|          0.0|             Green Alert|
|Illinois|

# End of analysis